In [1]:
import requests
import config
import pandas as pd
import numpy as np
from datetime import date
import pickle
import time
import sqlite3

In [2]:
URL = {
    'base': 'https://{proxy}.api.riotgames.com/lol/{url}',
    'summonerData': 'summoner/v{version}/summoners/by-name/{sumName}?api_key={apiKey}',
    'leagueData': 'league/v{version}/entries/{queueType}/{tier}/{rank}?api_key={apiKey}'
}

VERSION = {
    'summonerData': '4',
    'leagueData': '4'
}

In [3]:
queueTypes = ['RANKED_SOLO_5x5', 'RANKED_FLEX_SR', 'RANKED_FLEX_TT']
tiers = {'DIAMOND': 1, 'PLATINUM': 2, 'GOLD': 3, 'SILVER': 4, 'BRONZE': 5, 'IRON': 6}
ranks = {'I': 1, 'II': 2, 'III': 3, 'IV': 4}

In [4]:
class riotApi(object):
    
    def __init__(self, apiKey, region = 'na1'):
        self.apiKey = apiKey
        self.region = region
        
    def _request(self, apiUrl, params = {}):
        args = {'apiKey': self.apiKey}
        for key, value in params.items():
            if key not in args:
                args[key] = value
        response = requests.get(
            URL['base'].format(
                proxy = self.region,
                url = apiUrl),
            params = args)
        return response.json()
    
    def _checkConnection(self):
        pass
    
    def summonerData(self, name):
        apiUrl = URL['summonerData'].format(
            version = VERSION['summonerData'],
            sumName = name,
            apiKey = self.apiKey)
        return self._request(apiUrl)
    
    def leagueData(self, queueType, tier, rank):
        apiUrl = URL['leagueData'].format(
            version = VERSION['leagueData'],
            queueType = queueType,
            tier = tier,
            rank = rank,
            apiKey = self.apiKey)
        return self._request(apiUrl)

In [5]:
class sqlDatabase():
    
    def __init__(self, path):
        self.path = path
    
    def _open(self):
        self.db = sqlite3.connect(self.path)
        self.cursor = self.db.cursor()
        
    def _close(self):
        self.cursor.close()
        self.db.close()
        
    def appendData(self, df, table):
        self._open()
        
        try:
            # Load data from sql
            query = pd.read_sql_query(f"SELECT * FROM {table};", self.db)

            # Merge old with new data and remove duplicates
            df = query.append(df, ignore_index = True)
            print(f"{sum(df.duplicated())} duplicates deleted...")
            df.drop_duplicates(ignore_index = True, inplace = True)
        
        except:
            pass
        
        # Add scraped data to the database
        df.to_sql(table, self.db, if_exists = 'replace', index = False)
        
        self._close()
        
    def loadData(self, table):
        self._open()
        
        df = pd.read_sql_query(f"SELECT * FROM {table};", self.db)
        
        self._close()
        return df

In [6]:
# Configure the Riot API
api = riotApi(config.apiKey)

In [7]:
test = api.leagueData(queueTypes[0], 'DIAMOND', 'I')

In [8]:
test

{'status': {'message': 'Forbidden', 'status_code': 403}}

In [30]:
# Request summoner data and upload the data into the database
leagueRequest = []
for tierKey, tierVal in tier.items():
    for rankKey, rankVal in rank.items():
        leagueRequest += api.leagueData(queueTypes[0], tierKey, rankKey)

summonerDF = pd.DataFrame(leagueRequest)
summonerDF['date'] = date.today()

for key, val in ranks.items():
    summonerDF.loc[summonerDF['rank'] == key, 'rank'] = val
    
for key, val in tiers.items():
    summonerDF.loc[summonerDF['tier'] == key, 'tier'] = val

summonerDF = summonerDF.drop(columns = ['miniSeries'])

db_lol = sqlDatabase('../data/test.db')
db_lol.appendData(summonerDF, 'SUMMONERS')